In [16]:
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import os, random
import numpy as np
#from SeversonReproduced import select_dataset

In [17]:
testfile = pathlib.Path(r"C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData") / "FastCharge_000022_CH24_structure.json"
testfile.is_file()

True

In [4]:
df = pd.read_json(testfile)

In [124]:
df.head(5)

,@module,@class,barcode,protocol,channel_id,summary,cycles_interpolated,diagnostic_summary,diagnostic_interpolated,@version
cycle_index,beep.structure,ProcessedCyclerRun,el150800460492,2017-06-30_tests\20170630-5_2C_66per_3_5C.sdu,34,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,2020.10.19.20-8667111
discharge_capacity,beep.structure,ProcessedCyclerRun,el150800460492,2017-06-30_tests\20170630-5_2C_66per_3_5C.sdu,34,"[1.9506046000000001, 1.0649502, 1.068850899999...","[0.6424517913510791, 1.563939370206618, 1.5638...",NaN,NaN,2020.10.19.20-8667111
charge_capacity,beep.structure,ProcessedCyclerRun,el150800460492,2017-06-30_tests\20170630-5_2C_66per_3_5C.sdu,34,"[1.438199, 1.0652034, 1.0684291, 1.0701718, 1....","[0.16839407191795203, 1.0716072, 1.0716072, 1....",NaN,NaN,2020.10.19.20-8667111
discharge_energy,beep.structure,ProcessedCyclerRun,el150800460492,2017-06-30_tests\20170630-5_2C_66per_3_5C.sdu,34,"[6.1727977, 3.239139299999999, 3.2487645, 3.25...",NaN,NaN,NaN,2020.10.19.20-8667111
charge_energy,beep.structure,ProcessedCyclerRun,el150800460492,2017-06-30_tests\20170630-5_2C_66per_3_5C.sdu,34,"[4.7458897, 3.7140138, 3.7171833999999997, 3.7...",NaN,NaN,NaN,2020.10.19.20-8667111


In [18]:
def select_dataset(df, column):
    # tested for summary column 
    
    if not isinstance(column, (list, tuple)):
        column = [column]
        
    s = df.loc[:, column]
    print(f"Shape of selected packed dataset: {s.shape}")
    
    s = s.dropna()
    print(f"Shape of selected packed dataset without NaNs: {s.shape}")
    if s.empty:
        print("Non values found")
        return
    
    s = s.T.apply(pd.Series.explode).set_index("cycle_index")
    print(f"Shape of selected unpacked dataset: {s.shape}")

    return s

In [123]:
%%time

directory = r"C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData"
savepath1 = r"C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\ML_Github\data\interim\summary"
savepath2 = r"C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\ML_Github\data\interim\cycles_interpolated"

# iterate over files in that directory
i = 0
for filename in os.listdir(directory)[101:]:
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        # Make dataframe of file and add the summary column to list
        df = pd.read_json(f)
        newfilename = filename[0:-15]
        
        # Save csv file with summary data
        summary = select_dataset(df, 'summary')
        summary.to_csv(os.path.join(savepath1, newfilename + '_summary.csv'))
        
        # Save csv file with Delta Q100-10 data
        cycles_interpolated = select_dataset(df, 'cycles_interpolated')
        Qd_c10 = cycle10.loc[cycle10['step_type'] == 'discharge']['discharge_capacity']
        Qd_c100 = cycle100.loc[cycle100['step_type'] == 'discharge']['discharge_capacity']
        DeltaQ100_10 = pd.DataFrame(Qd_c100.values - Qd_c10.values)
        DeltaQ100_10.to_csv(os.path.join(savepath2, newfilename + '_DeltaQ100_10.csv'), index=False, header=False)

C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData\FastCharge_000045_CH3_structure.json
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (16, 1)
Shape of selected unpacked dataset: (1177, 15)
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (8, 1)
Shape of selected unpacked dataset: (2355000, 7)
C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData\FastCharge_000046_CH31_structure.json
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (16, 1)
Shape of selected unpacked dataset: (891, 15)
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (8, 1)
Shape of selected unpacked dataset: (1784000, 7)
C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData\FastCharge_000046_CH32_structure.json
Shape of selected packed dataset: (21, 1)
Shap

Shape of selected unpacked dataset: (1304000, 7)
C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData\FastCharge_000059_CH33_structure.json
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (16, 1)
Shape of selected unpacked dataset: (546, 15)
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (8, 1)
Shape of selected unpacked dataset: (1092000, 7)
C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData\FastCharge_000060_CH29_structure.json
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (16, 1)
Shape of selected unpacked dataset: (522, 15)
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (8, 1)
Shape of selected unpacked dataset: (1044000, 7)
C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData\FastCharge_000061_CH43_structure.js